In [1]:
import sys
sys.path.insert(0, 'cucumber_data_report/import file')

import model_import_file
import evaluation_metrics_import_file

2022-05-01 13:24:30.956655: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.0/lib64
2022-05-01 13:24:30.956697: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
path_to_data = "dataset"
path_to_save = "cucumber_data_report/data_save_with_aug/"

In [3]:
import matplotlib
matplotlib.rcParams.update({'font.size': 23})

In [4]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import time

import tensorflow as tf
from keras.models import model_from_json
import json,codecs

def saveHist(path,history):
    
    new_hist = {}
    for key in list(history.history.keys()):
        new_hist[key]=history.history[key]
        if type(history.history[key]) == np.ndarray:
            new_hist[key] = history.history[key].tolist()
        elif type(history.history[key]) == list:
            if  type(history.history[key][0]) == np.float64:
                new_hist[key] = list(map(float, history.history[key]))
            
    print(new_hist)
    with codecs.open(path, 'w', encoding='utf-8') as file:
        json.dump(new_hist, file, separators=(',', ':'), sort_keys=True, indent=4) 

def loadHist(path):
    with codecs.open(path, 'r', encoding='utf-8') as file:
        n = json.loads(file.read())
    return n

class_dict = {}

counter = 0
for folder in os.listdir(path_to_data):
    path = os.path.join(path_to_data,folder)
    for subfolder in os.listdir(path):
        class_dict[subfolder] = counter
        counter += 1
    break

print(counter)
class_dict

class_list = []
for ele in class_dict:
    class_list.append(ele)

class_list

import random

def image_augmentation(img):
    central_cropped_image = tf.image.central_crop(img_array, central_fraction=0.9)
    random_left_right_flip = tf.image.stateless_random_flip_left_right(img,(2,3))
    random_up_down_flip = tf.image.stateless_random_flip_up_down(img,(2,3))
    random_contrast = tf.image.stateless_random_contrast(img,0.5, 0.9,(2,3))
    random_saturation = tf.image.stateless_random_saturation(img,0.5, 0.9,(2,3))
    random_brightness = tf.image.stateless_random_brightness(img,0.2,(2,3))
        
    imgs = [central_cropped_image, random_left_right_flip, random_up_down_flip, random_contrast, random_saturation, random_brightness]
    return imgs


IMG_SIZE_X = 200
IMG_SIZE_Y = 200

total_training_images_classwise = {}
total_testing_images_classwise ={}

X_train = []
y_train = []

X_test = []
y_test = []

for folder in os.listdir(path_to_data):
    path = os.path.join(path_to_data,folder)
    for subfolder in os.listdir(path):
        path_to_subfolder = os.path.join(path,subfolder)
        key = subfolder
        number_of_image_in_folder = len(os.listdir(path_to_subfolder))
        if folder == "testing":
            total_testing_images_classwise[subfolder] = number_of_image_in_folder
        else:
            total_training_images_classwise[subfolder] = 6*number_of_image_in_folder
        for img in os.listdir(path_to_subfolder):
            try:
                img_array = cv2.imread(os.path.join(path_to_subfolder,img))
                
                if folder == "testing":
                    resized_array = cv2.resize(img_array, (IMG_SIZE_X,IMG_SIZE_Y))
                    X_test.append(resized_array)
                    y_test.append(class_dict[subfolder])
                else:
                    augmented_image = image_augmentation(img_array)
                    for imgs in augmented_image:
                      numpy_image = imgs.numpy().astype("uint8")
                      resized_array = cv2.resize(numpy_image, (IMG_SIZE_X,IMG_SIZE_Y))
                      X_train.append(resized_array)
                      y_train.append(class_dict[subfolder])
            except Exception as e:
                print("Exception is : "+e)
                pass
            
print("Size of X_train : " + str(len(X_train)))
print("Size of y_train : " + str(len(y_train)))
print("Size of X_test : " + str(len(X_test)))
print("Size of y_test : " + str(len(y_test)))

file_path = path_to_save
evaluation_metrics_import_file.save_histogram_for_training_and_testing_data(class_list,total_training_images_classwise,total_testing_images_classwise,file_path+"histogram_for_training_and_testing_data.jpg")

X_train_numpy_array = np.array(X_train)
y_train = np.array(y_train)
X_test_numpy_array = np.array(X_test)
y_test = np.array(y_test)

X_train_numpy_array = X_train_numpy_array/255
X_test_numpy_array = X_test_numpy_array/255

input_shape = IMG_SIZE_X, IMG_SIZE_Y, 3
n_classes = counter
epoch = 50
# Adagrad
adagrad_opt = tf.keras.optimizers.Adagrad(
    learning_rate=0.0005,
    initial_accumulator_value=0.1,
    epsilon=1e-07,
    name="Adagrad",
)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)

model_history = []

model_name = "proposed_model_1"
print("Proposed Model")
if os.path.exists(path_to_save + model_name + '.json'):
    json_file = open(path_to_save + model_name + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    proposed_model_1 = model_from_json(loaded_model_json)
    # load weights into new model
    proposed_model_1.load_weights(path_to_save + model_name + ".h5")
    print("Loaded model from disk")
    proposed_model_1.compile(optimizer=adagrad_opt,loss = loss , metrics=['accuracy'])
    # Load History
    proposed_model_1_history = loadHist(path_to_save + model_name + "_history.json")
else:
    proposed_model_1 = model_import_file.proposed_model_1(input_shape,n_classes)
    proposed_model_1.compile(optimizer=adagrad_opt,loss = loss , metrics=['accuracy'])
    proposed_model_1_history = proposed_model_1.fit(X_train_numpy_array,y_train,batch_size = 4,epochs=epoch)

    # Save Model
    model_json = proposed_model_1.to_json()
    with open(path_to_save + model_name + ".json", "w") as json_file:
        json_file.write(model_json)
    proposed_model_1.save_weights(path_to_save + model_name + ".h5")
    # Save History
    saveHist(path_to_save + model_name + "_history.json",proposed_model_1_history)
    print("Saved model to disk")

proposed_model_1_eval = proposed_model_1.evaluate(X_test_numpy_array,y_test)

model_history.append(proposed_model_1_history)

file_path = path_to_save
model_name = "proposed_model_1"
evaluation_metrics_import_file.save_conf_matrix(proposed_model_1,X_test_numpy_array,y_test,class_list,file_path+model_name+"conf_matrix.jpg")
evaluation_metrics_import_file.print_evaluation_metrics(proposed_model_1,X_test_numpy_array,y_test,file_path+model_name+"evaluation_metrics.xlsx")

# For deleting files if not wanted
# model_name = "proposed_model_1"
want_to_delete = False
if want_to_delete:
    os.remove(path_to_save + model_name + '.json')
    os.remove(path_to_save + model_name + '.h5')
    os.remove(path_to_save + model_name + "history.json")

model_name = "soyNet"
print("Soynet Model")
if os.path.exists(path_to_save + model_name + '.json'):
    json_file = open(path_to_save + model_name + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    soyNet_model = model_from_json(loaded_model_json)
    # load weights into new model
    soyNet_model.load_weights(path_to_save + model_name + ".h5")
    print("Loaded model from disk")
    soyNet_model.compile(optimizer=adagrad_opt,loss = loss , metrics=['accuracy'])
    # Load History
    soyNet_model_history = loadHist(path_to_save + model_name + "_history.json")
else:
    soyNet_model = model_import_file.soyNet(input_shape,n_classes)
    soyNet_model.compile(optimizer=adagrad_opt,loss = loss , metrics=['accuracy'])
    soyNet_model_history = soyNet_model.fit(X_train_numpy_array,y_train,batch_size = 4,epochs=epoch)

    # Save Model
    model_json = soyNet_model.to_json()
    with open(path_to_save + model_name + ".json", "w") as json_file:
        json_file.write(model_json)
    soyNet_model.save_weights(path_to_save + model_name + ".h5")
    # Save History
    saveHist(path_to_save + model_name + "_history.json",soyNet_model_history)
    print("Saved model to disk")

soyNet_model_eval = soyNet_model.evaluate(X_test_numpy_array,y_test)

# For deleting files if not wanted
model_name = "soyNet"
want_to_delete = False
if want_to_delete:
  os.remove(path_to_save + model_name + '.json')
  os.remove(path_to_save + model_name + '.h5')
  os.remove(path_to_save + model_name + "history.json")

model_history.append(soyNet_model_history)

file_path = path_to_save
model_name = "soyNet"
evaluation_metrics_import_file.save_conf_matrix(soyNet_model,X_test_numpy_array,y_test,class_list,file_path+model_name+"conf_matrix.jpg")
evaluation_metrics_import_file.print_evaluation_metrics(soyNet_model,X_test_numpy_array,y_test,file_path+model_name+"evaluation_metrics.xlsx")

model_name = "xception_network"
print("Xception Network Model")
if os.path.exists(path_to_save + model_name + '.json'):
    json_file = open(path_to_save + model_name + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    xception_model = model_from_json(loaded_model_json)
    # load weights into new model
    xception_model.load_weights(path_to_save + model_name + ".h5")
    print("Loaded model from disk")
    xception_model.compile(optimizer=adagrad_opt,loss = loss , metrics=['accuracy'])
    # Load History
    xception_model_history = loadHist(path_to_save + model_name + "_history.json")
else:
    xception_model = model_import_file.xception_network(input_shape,n_classes)
    xception_model.compile(optimizer=adagrad_opt,loss = loss , metrics=['accuracy'])
    xception_model_history = xception_model.fit(X_train_numpy_array,y_train,batch_size = 4,epochs=epoch)

    # Save Model
    model_json = xception_model.to_json()
    with open(path_to_save + model_name + ".json", "w") as json_file:
        json_file.write(model_json)
    xception_model.save_weights(path_to_save + model_name + ".h5")
    # Save History
    saveHist(path_to_save + model_name + "_history.json",xception_model_history)
    print("Saved model to disk")

xception_model_eval = xception_model.evaluate(X_test_numpy_array,y_test)

model_history.append(xception_model_history)

file_path = path_to_save
model_name = "xception_network"
evaluation_metrics_import_file.save_conf_matrix(xception_model,X_test_numpy_array,y_test,class_list,file_path+model_name+"conf_matrix.jpg")
evaluation_metrics_import_file.print_evaluation_metrics(xception_model,X_test_numpy_array,y_test,file_path+model_name+"evaluation_metrics.xlsx")

model_name = "densenet_121_network"
print("DenseNet Model")
if os.path.exists(path_to_save + model_name + '.json'):
    json_file = open(path_to_save + model_name + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    densenet_121_model = model_from_json(loaded_model_json)
    # load weights into new model
    densenet_121_model.load_weights(path_to_save + model_name + ".h5")
    print("Loaded model from disk")
    densenet_121_model.compile(optimizer=adagrad_opt,loss = loss , metrics=['accuracy'])
    # Load History
    densenet_121_model_history = loadHist(path_to_save + model_name + "_history.json")
else:
    densenet_121_model = model_import_file.densenet_121_network(input_shape,n_classes)
    densenet_121_model.compile(optimizer=adagrad_opt,loss = loss , metrics=['accuracy'])
    densenet_121_model_history = densenet_121_model.fit(X_train_numpy_array,y_train,batch_size = 4,epochs=epoch)

    # Save Model
    model_json = densenet_121_model.to_json()
    with open(path_to_save + model_name + ".json", "w") as json_file:
        json_file.write(model_json)
    densenet_121_model.save_weights(path_to_save + model_name + ".h5")
    # Save History
    saveHist(path_to_save + model_name + "_history.json",densenet_121_model_history)
    print("Saved model to disk")

densenet_121_model_eval = densenet_121_model.evaluate(X_test_numpy_array,y_test)

model_history.append(densenet_121_model_history)

file_path = path_to_save
model_name = "densenet_121_network"
evaluation_metrics_import_file.save_conf_matrix(densenet_121_model,X_test_numpy_array,y_test,class_list,file_path+model_name+"conf_matrix.jpg")
evaluation_metrics_import_file.print_evaluation_metrics(densenet_121_model,X_test_numpy_array,y_test,file_path+model_name+"evaluation_metrics.xlsx")

model_name = "alexnet"
print("Alexnet Model")
if os.path.exists(path_to_save + model_name + '.json'):
    json_file = open(path_to_save + model_name + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    alexnet_model = model_from_json(loaded_model_json)
    # load weights into new model
    alexnet_model.load_weights(path_to_save + model_name + ".h5")
    print("Loaded model from disk")
    alexnet_model.compile(optimizer=adagrad_opt,loss = loss , metrics=['accuracy'])
    # Load History
    alexnet_model_history = loadHist(path_to_save + model_name + "_history.json")
else:
    alexnet_model = model_import_file.alexnet(input_shape,n_classes)
    alexnet_model.compile(optimizer=adagrad_opt,loss = loss , metrics=['accuracy'])
    alexnet_model_history = alexnet_model.fit(X_train_numpy_array,y_train,batch_size = 4,epochs=epoch)

    # Save Model
    model_json = alexnet_model.to_json()
    with open(path_to_save + model_name + ".json", "w") as json_file:
        json_file.write(model_json)
    alexnet_model.save_weights(path_to_save + model_name + ".h5")
    # Save History
    saveHist(path_to_save + model_name + "_history.json",alexnet_model_history)
    print("Saved model to disk")

alexnet_model_eval = alexnet_model.evaluate(X_test_numpy_array,y_test)

model_history.append(alexnet_model_history)

file_path = path_to_save
model_name = "alexnet"
evaluation_metrics_import_file.save_conf_matrix(alexnet_model,X_test_numpy_array,y_test,class_list,file_path+model_name+"conf_matrix.jpg")
evaluation_metrics_import_file.print_evaluation_metrics(alexnet_model,X_test_numpy_array,y_test,file_path+model_name+"evaluation_metrics.xlsx")

model_name = "pretrained_VGG16"
print("VGG-16 Model")
if os.path.exists(path_to_save + model_name + '.json'):
    json_file = open(path_to_save + model_name + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    pretrained_VGG16_model = model_from_json(loaded_model_json)
    # load weights into new model
    pretrained_VGG16_model.load_weights(path_to_save + model_name + ".h5")
    print("Loaded model from disk")
    pretrained_VGG16_model.compile(optimizer=adagrad_opt,loss = loss , metrics=['accuracy'])
    # Load History
    pretrained_VGG16_model_history = loadHist(path_to_save + model_name + "_history.json")
else:
    pretrained_VGG16_model = model_import_file.pretrained_VGG16(input_shape,n_classes)
    pretrained_VGG16_model.compile(optimizer=adagrad_opt,loss = loss , metrics=['accuracy'])
    pretrained_VGG16_model_history = pretrained_VGG16_model.fit(X_train_numpy_array,y_train,batch_size = 4,epochs=epoch)

    # Save Model
    model_json = pretrained_VGG16_model.to_json()
    with open(path_to_save + model_name + ".json", "w") as json_file:
        json_file.write(model_json)
    pretrained_VGG16_model.save_weights(path_to_save + model_name + ".h5")
    # Save History
    saveHist(path_to_save + model_name + "_history.json",pretrained_VGG16_model_history)
    print("Saved model to disk")

pretrained_VGG16_model_eval = pretrained_VGG16_model.evaluate(X_test_numpy_array,y_test)

model_history.append(pretrained_VGG16_model_history)

file_path = path_to_save
model_name = "pretrained_VGG16"
evaluation_metrics_import_file.save_conf_matrix(pretrained_VGG16_model,X_test_numpy_array,y_test,class_list,file_path+model_name+"conf_matrix.jpg")
evaluation_metrics_import_file.print_evaluation_metrics(pretrained_VGG16_model,X_test_numpy_array,y_test,file_path+model_name+"evaluation_metrics.xlsx")

model_name = "pretrained_ResNet50"
print("Resnet Model")
if os.path.exists(path_to_save + model_name + '.json'):
    json_file = open(path_to_save + model_name + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    pretrained_ResNet50_model = model_from_json(loaded_model_json)
    # load weights into new model
    pretrained_ResNet50_model.load_weights(path_to_save + model_name + ".h5")
    print("Loaded model from disk")
    pretrained_ResNet50_model.compile(optimizer=adagrad_opt,loss = loss , metrics=['accuracy'])
    # Load History
    pretrained_ResNet50_model_history = loadHist(path_to_save + model_name + "_history.json")
else:
    pretrained_ResNet50_model = model_import_file.pretrained_ResNet50(input_shape,n_classes)
    pretrained_ResNet50_model.compile(optimizer=adagrad_opt,loss = loss , metrics=['accuracy'])
    pretrained_ResNet50_model_history = pretrained_ResNet50_model.fit(X_train_numpy_array,y_train,batch_size = 4,epochs=epoch)

    # Save Model
    model_json = pretrained_ResNet50_model.to_json()
    with open(path_to_save + model_name + ".json", "w") as json_file:
        json_file.write(model_json)
    pretrained_ResNet50_model.save_weights(path_to_save + model_name + ".h5")
    # Save History
    saveHist(path_to_save + model_name + "_history.json",pretrained_ResNet50_model_history)
    print("Saved model to disk")

pretrained_ResNet50_model_eval = pretrained_ResNet50_model.evaluate(X_test_numpy_array,y_test)

model_history.append(pretrained_ResNet50_model_history)

file_path = path_to_save
model_name = "pretrained_ResNet50"
evaluation_metrics_import_file.save_conf_matrix(pretrained_ResNet50_model,X_test_numpy_array,y_test,class_list,file_path+model_name+"conf_matrix.jpg")
evaluation_metrics_import_file.print_evaluation_metrics(pretrained_ResNet50_model,X_test_numpy_array,y_test,file_path+model_name+"evaluation_metrics.xlsx")

model_name = "pretrained_EfficientNetB2"
print("Efficient-Net B2 Model")
if os.path.exists(path_to_save + model_name + '.json'):
    json_file = open(path_to_save + model_name + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    pretrained_EfficientNetB2_model = model_from_json(loaded_model_json)
    # load weights into new model
    pretrained_EfficientNetB2_model.load_weights(path_to_save + model_name + ".h5")
    print("Loaded model from disk")
    pretrained_EfficientNetB2_model.compile(optimizer=adagrad_opt,loss = loss , metrics=['accuracy'])
    # Load History
    pretrained_EfficientNetB2_model_history = loadHist(path_to_save + model_name + "_history.json")
else:
    pretrained_EfficientNetB2_model = model_import_file.pretrained_EfficientNetB2(input_shape,n_classes)
    pretrained_EfficientNetB2_model.compile(optimizer=adagrad_opt,loss = loss , metrics=['accuracy'])
    pretrained_EfficientNetB2_model_history = pretrained_EfficientNetB2_model.fit(X_train_numpy_array,y_train,batch_size = 4,epochs=epoch)

    # Save Model
    model_json = pretrained_EfficientNetB2_model.to_json()
    with open(path_to_save + model_name + ".json", "w") as json_file:
        json_file.write(model_json)
    pretrained_EfficientNetB2_model.save_weights(path_to_save + model_name + ".h5")
    # Save History
    saveHist(path_to_save + model_name + "_history.json",pretrained_EfficientNetB2_model_history)
    print("Saved model to disk")

pretrained_EfficientNetB2_model_eval = pretrained_EfficientNetB2_model.evaluate(X_test_numpy_array,y_test)

model_history.append(pretrained_EfficientNetB2_model_history)

file_path = path_to_save
model_name = "pretrained_EfficientNetB2"
evaluation_metrics_import_file.save_conf_matrix(pretrained_EfficientNetB2_model,X_test_numpy_array,y_test,class_list,file_path+model_name+"conf_matrix.jpg")
evaluation_metrics_import_file.print_evaluation_metrics(pretrained_EfficientNetB2_model,X_test_numpy_array,y_test,file_path+model_name+"evaluation_metrics.xlsx")

evaluation_metrics_import_file.save_of_training_accuracy_vs_epochs(model_history,epoch,file_path+"training_accuracy_vs_epochs")
evaluation_metrics_import_file.save_of_training_loss_vs_epochs(model_history,epoch,file_path+"training_loss_vs_epochs")

evaluation_metrics_import_file.func_help()

training_accuracy_of_different_models = []
testing_accuracy_of_different_models = []
model_list = ["Proposed Model 1","SoyNet Model","Xception Model","Densenet 121 Model","AlexNet Model","VGG16 Model","ResNet 50 Model","EfficientNetB2 Model"]

sz = len(model_history)
for i in range(sz):
    length = len(model_history[i]['accuracy'])
    training_accuracy_of_different_models.append(model_history[i]['accuracy'][length-1])


testing_accuracy_of_different_models.append(proposed_model_1_eval[1])
testing_accuracy_of_different_models.append(soyNet_model_eval[1])
testing_accuracy_of_different_models.append(xception_model_eval[1])
testing_accuracy_of_different_models.append(densenet_121_model_eval[1])
testing_accuracy_of_different_models.append(alexnet_model_eval[1])
testing_accuracy_of_different_models.append(pretrained_VGG16_model_eval[1])
testing_accuracy_of_different_models.append(pretrained_ResNet50_model_eval[1])
testing_accuracy_of_different_models.append(pretrained_EfficientNetB2_model_eval[1])

evaluation_metrics_import_file.save_histogram_for_training_and_testing_accuracy(model_list,training_accuracy_of_different_models,testing_accuracy_of_different_models,file_path+"training_and_testing_accuracy.jpg")



2


2022-05-01 13:24:34.387207: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sachin/anaconda3/envs/aman/lib/python3.7/site-packages/cv2/../../lib64:/usr/local/cuda-10.0/lib64
2022-05-01 13:24:34.387321: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sachin/anaconda3/envs/aman/lib/python3.7/site-packages/cv2/../../lib64:/usr/local/cuda-10.0/lib64
2022-05-01 13:24:34.387413: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sachin/anaconda3/envs/aman/lib/python3.7/site-

Size of X_train : 2814
Size of y_train : 2814
Size of X_test : 222
Size of y_test : 222
Proposed Model
Loaded model from disk


/home/sachin/anaconda3/envs/aman/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


7/7 [==============================] - 9s 686ms/step - loss: 0.1779 - accuracy: 0.9234


 Report 



+-----------+-----------+----------------------------------+
|   Class 1 |   Class 2 | Measure                          |
+===========+===========+==================================+
| 92.3423   | 92.3423   | Accuracy                         |
+-----------+-----------+----------------------------------+
|  7.65766  |  7.65766  | Error                            |
+-----------+-----------+----------------------------------+
|  0.981818 |  0.866071 | Precision                        |
+-----------+-----------+----------------------------------+
|  0.878049 |  0.979798 | Recall                           |
+-----------+-----------+----------------------------------+
|  0.878049 |  0.979798 | Sensitivity                      |
+-----------+-----------+----------------------------------+
|  0.979798 |  0.878049 | Specificity                      |
+-----------+-----------+--------------------

...

7/7 [==============================] - 7s 730ms/step - loss: 0.6903 - accuracy: 0.5495


 Report 



+-----------+-----------+----------------------------------+
|   Class 1 |   Class 2 | Measure                          |
+===========+===========+==================================+
| 54.955    | 54.955    | Accuracy                         |
+-----------+-----------+----------------------------------+
| 45.045    | 45.045    | Error                            |
+-----------+-----------+----------------------------------+
|  0.755556 |  0.497175 | Precision                        |
+-----------+-----------+----------------------------------+
|  0.276423 |  0.888889 | Recall                           |
+-----------+-----------+----------------------------------+
|  0.276423 |  0.888889 | Sensitivity                      |
+-----------+-----------+----------------------------------+
|  0.888889 |  0.276423 | Specificity                      |
+-----------+-----------+--------------------

/home/sachin/anaconda3/envs/aman/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


7/7 [==============================] - 6s 484ms/step - loss: 0.6973 - accuracy: 0.4459


cucumber_data_report/import file/evaluation_metrics_import_file.py:82: RuntimeWarning: invalid value encountered in true_divide
  prec = (TP) / (TP + FP)
cucumber_data_report/import file/evaluation_metrics_import_file.py:111: RuntimeWarning: divide by zero encountered in true_divide
  return num/(np.sqrt(den))




 Report 



+-----------+-------------+----------------------------------+
|   Class 1 |     Class 2 | Measure                          |
+===========+=============+==================================+
|   44.5946 |   44.5946   | Accuracy                         |
+-----------+-------------+----------------------------------+
|   55.4054 |   55.4054   | Error                            |
+-----------+-------------+----------------------------------+
|  nan      |    0.445946 | Precision                        |
+-----------+-------------+----------------------------------+
|    0      |    1        | Recall                           |
+-----------+-------------+----------------------------------+
|    0      |    1        | Sensitivity                      |
+-----------+-------------+----------------------------------+
|    1      |    0        | Specificity                      |
+-----------+-------------+----------------------------------+
|  nan      |    0.616822 | F1 Score     